<a target="_blank" href="../cluster" style="font-size:20px">All Applications (YARN)</a>

# Домашнее задание

_Это задание очень похоже на задание, которое вы уже делали на Hadoop, так и задумано, оцените, насколько проще оно решается на Spark._

Будем использовать логи прослушивания музыкальных исполнителей в сервисе Яндекс.Музыка.

Файл `events.csv` содержит записи вида `Пользователь,Исполнитель,Число прослушиваний,Число пропусков`:
```csv
userId,artistId,plays,skips
0,335,1,0
0,708,1,0
0,710,2,1
0,815,1,1
```

Вам необходимо проделать следующее:
1. **Оставьте в данных только тех пользователей, для которых сумма plays строго больше 2000. Сколько таких пользователей?**
2. **В отфильтрованных на первом шаге данных найдите 5 самых популярных по числу пользователей исполнителей (идентификаторы).**

Детали:
1. Давайте считать, что список прослушиваний одного пользователя всегда помещается в память.

Решение сохраните в файл `result.json`. Пример содержимого файла:

```json
{
    "q1": 123,
    "q2": [
        4,
        5,
        6,
        7,
        8
    ]
}```

In [62]:
# пример содержимого файла
! head -n 5 yandex_music/events.csv

userId,artistId,plays,skips
0,335,1,0
0,708,1,0
0,710,2,1
0,815,1,1


In [63]:
# копируем файлы в HDFS
! hadoop fs -copyFromLocal yandex_music /
! hadoop fs -ls -h /yandex_music

copyFromLocal: `/yandex_music/README.txt': File exists
copyFromLocal: `/yandex_music/artists.jsonl': File exists
copyFromLocal: `/yandex_music/events.csv': File exists
Found 3 items
-rw-r--r--   1 jovyan supergroup        254 2021-08-11 12:11 /yandex_music/README.txt
-rw-r--r--   1 jovyan supergroup      3.7 M 2021-08-11 12:11 /yandex_music/artists.jsonl
-rw-r--r--   1 jovyan supergroup     47.6 M 2021-08-11 12:11 /yandex_music/events.csv


In [3]:
# https://spark.apache.org/docs/latest/rdd-programming-guide.html
# http://spark.apache.org/docs/latest/sql-getting-started.html

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [64]:
# загружаем csv как Spark DataFrame
events = se.read.csv("hdfs:///yandex_music/events.csv", header=True)  # в первой строке у нас заголовок
events.registerTempTable("events")
events.limit(5).toPandas()

,userId,artistId,plays,skips
0,0,335,1,0
1,0,708,1,0
2,0,710,2,1
3,0,815,1,1
4,0,880,1,1


In [5]:
# можно сконвертировать этот DataFrame в RDD
events.rdd.take(5)

[Row(userId='0', artistId='335', plays='1', skips='0'),
 Row(userId='0', artistId='708', plays='1', skips='0'),
 Row(userId='0', artistId='710', plays='2', skips='1'),
 Row(userId='0', artistId='815', plays='1', skips='1'),
 Row(userId='0', artistId='880', plays='1', skips='1')]

In [6]:
# ТУТ ВАШ КОД

In [87]:
q1 = se.sql("""
select count(*)
from (select 
    userId,
    sum(plays) as sum
from events
group by userId)
where sum > 2000
""").toPandas()
q1

,count(1)
0,1705


In [73]:
q2 = se.sql("""
select artistId, count(artistId)
from events e 
    join(
        select 
            userId,
            sum(plays) as sum
        from events
        group by userId
) s on e.userId = s.userId
where sum > 2000
group by artistId
order by count(artistId) DESC
limit 5
""").toPandas()
q2

,artistId,count(artistId)
0,11368,1421
1,3629,1274
2,259,1221
3,44148,1191
4,23524,1167


In [88]:
q1.values[0][0]

1705

In [81]:
q2.artistId.values

array(['11368', '3629', '259', '44148', '23524'], dtype=object)

In [89]:
%%file result.json
{
    "q1": 1705,
    "q2": [
        11368,
        3629,
        259,
        44148,
        23524
    ]
}

Overwriting result.json


In [90]:
# останавливаем Spark (и YARN приложение)
# sc.stop()